In [1]:
import os
import sys
import io
import json
import boto3
import numpy as np
import pandas as pd
from PIL import Image
from dotenv import load_dotenv
import base64
import requests


In [7]:
load_dotenv()
sys.path.append("../")

In [8]:
INFERENCE_PROFILE_ID = os.getenv("INFERENCE_PROFILE_ID")


In [ ]:
# AWS S3 setup
s3 = boto3.client('s3',region_name='eu-west-3')
bucket_name = #bucketname
image_folder = 'Data/train/images/' 
csv_path = 'Data/train/train_images.csv'  


In [4]:
def load_csv_from_s3(bucket_name, csv_key):
    response = s3.get_object(Bucket=bucket_name, Key=csv_key)
    return pd.read_csv(io.BytesIO(response['Body'].read()))

In [5]:
df = load_csv_from_s3(bucket_name, csv_path)
image_paths = df['imagePath'].tolist()
captions = df['caption'].tolist()

In [3]:
Bedrock = boto3.client(
    'bedrock-runtime',
    region_name='eu-west-3',  
)

In [4]:
def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

image_base64 = encode_image_to_base64("C:/Users/14384/STAGE/Data/train/images/image_10007.jpg")

In [5]:
payload = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": image_base64
                    }
                },
                {
                    "type": "text",
                    "text": "Descrive this medical image"
                }
            ]
        }
    ],
    "max_tokens": 1000,
    "anthropic_version": "bedrock-2023-05-31"
}


In [9]:
response = Bedrock.invoke_model(
    modelId=INFERENCE_PROFILE_ID,
    contentType="application/json",
    body=json.dumps(payload)
)


In [10]:
output_binary = response["body"].read()
output_json = json.loads(output_binary)
output = output_json["content"][0]["text"]


In [11]:
print(output)


This image appears to be a CT (computed tomography) scan of the abdomen, showing a cross-sectional view of the body. The scan is in black and white, with various shades of gray indicating different tissue densities.

In this image, we can see:

1. The liver, which takes up a large portion of the right side of the image. It appears as a relatively uniform gray area.

2. The stomach, visible on the left side, containing some darker areas which could indicate the presence of air or fluid.

3. The spine, visible at the bottom center of the image, appearing as a bright white structure.

4. Several other abdominal organs and structures, though without more context it's difficult to identify them all with certainty.

5. There appears to be some irregularity or abnormality in the liver, as evidenced by a large, darker area within the liver tissue. This could potentially indicate a mass, cyst, or other pathology, but a medical professional would need to interpret this finding.

6. The image is 